In [ ]:
import pandas as pd

In [ ]:
# FILLER
EQUITY_TYPE = 'despac'
FILTER_DAYS = 5
FILTER_PCT = 25
FILTER_DIRECTION = 'up'

In [ ]:
stock_data = pd.read_csv(f'../data/unadjusted_stock_data/{EQUITY_TYPE}/stock_data.csv')
options_data = pd.read_parquet(f"../data/option_data/{EQUITY_TYPE}/clean_options_data.parquet")

earliest_date_per_ticker = (
    options_data
    .index
    .to_frame(index=False)
    .groupby("ticker")["date"]
    .min()
)

# Clean data
stock_data['date'] = pd.to_datetime(stock_data['date'])
stock_data['ticker'] = stock_data['ticker'].str.upper()
stock_data['close'] = pd.to_numeric(stock_data['close'], errors='coerce')

stock_data = stock_data.set_index(['date', 'ticker']).sort_index()
stock_data = stock_data.rename(columns={'close': 'stock_price'})

stock_data = stock_data.loc[
    stock_data.index.get_level_values("date")
    >= stock_data.index.get_level_values("ticker").map(earliest_date_per_ticker)
]

stock_data = stock_data.loc[
    stock_data.index.get_level_values("date")
    <= options_data.index.get_level_values("date").max()
]

stock_data.to_parquet(f'../data/stock_data/{EQUITY_TYPE}/clean_stock_data.parquet')

In [ ]:
stock_data